In [1]:
pip install rouge_score evaluate pyngrok 

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=07b8035b092cf1dd7c1df7171afe7e3b5f4d3a12a8af68339aee5182a9378cc2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [2]:
!curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
	| sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null \
	&& echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
	| sudo tee /etc/apt/sources.list.d/ngrok.list \
	&& sudo apt update \
	&& sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]          
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]           
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [66.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:9 https://ngrok-agent.s3.amazonaws.com buster/main amd64 Packages [7,389 B]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,309 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]m
Get:

In [3]:
!ngrok config add-authtoken 2sLV4iA8j8iRMAjLywMaB7vLkiD_2nEwWQKE4rAYsFw59Mg4v

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
from pyngrok import ngrok
from tensorboard import program

log_dir = '/kaggle/working/logs'  
tb = program.TensorBoard()
tb.configure(argv=[None, '--logdir', log_dir])

url = tb.launch()

public_url = ngrok.connect(6006)  
public_url


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784



<NgrokTunnel: "https://c078-34-105-107-239.ngrok-free.app" -> "http://localhost:6006">

In [2]:
from datasets import load_dataset, load_from_disk, DatasetDict

original_dataset = load_dataset("scientific_papers", "arxiv", trust_remote_code=True) 
original_dataset = original_dataset.remove_columns("section_names")
original_dataset

README.md:   0%|          | 0.00/8.27k [00:00<?, ?B/s]

scientific_papers.py:   0%|          | 0.00/5.35k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/203037 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6436 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6440 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 203037
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6436
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6440
    })
})

In [3]:
train_slice = int(len(original_dataset['train']) * 0.05)
train_dataset = original_dataset['train'].select(range(train_slice))
train_dataset

Dataset({
    features: ['article', 'abstract'],
    num_rows: 10151
})

In [4]:
import re

def replace_semicolon(text, threshold=10):
    new_text = ""
    for subset in re.split(';', text):
        subset = subset.strip() 
        if len(subset.split()) > threshold:
            new_text += ". " + subset[0].upper() + subset[1:]
        else:
            new_text += ", " + subset

    return new_text

PAREN_re = re.compile('\([^(]+\ [^\(]+\)')
BAD_PUNCT_RE = re.compile(r'([%s])' % re.escape('"#%&\*\+/<=>@[\]^{|}~_$'), re.UNICODE)
BULLET_RE = re.compile('\n[\ \t]*`*\([a-zA-Z0-9]*\)')
DASH_RE = re.compile('--+')
WHITESPACE_RE = re.compile('\s+')
EMPTY_SENT_RE = re.compile('[,\.]\ *[\.,]')
FIX_START_RE = re.compile('^[^A-Za-z]*')
FIX_PERIOD = re.compile('\.([A-Za-z])')
REF_RE = re.compile(r"@\w+|see e\.g\..*?(\.|$)|refer to.*?(\.|$)")
FIX_SPACES_AROUND_PUNCT = re.compile(r'\s+([.,!?;:])')
FIX_PARENTHESES = re.compile(r'\(\s*([^)]*?)\s*\)')

def clean_text(text):
    text = REF_RE.sub('', text)
    text = PAREN_re.sub('', text)
    text = BULLET_RE.sub(' ', text)
    text = text.replace('&lt;all&gt;', '')
    text = BAD_PUNCT_RE.sub('', text)
    text = DASH_RE.sub(' ', text)
    text = WHITESPACE_RE.sub(' ', text)
    text = EMPTY_SENT_RE.sub('.', text)
    text = replace_semicolon(text)
    text = FIX_START_RE.sub('', text)
    text = FIX_PERIOD.sub(". \g<1>", text)
    text = text.replace('``', '"')
    text = text.replace('\'\'', '"')
    text = FIX_SPACES_AROUND_PUNCT.sub(r'\1', text)
    text = FIX_PARENTHESES.sub(r'(\1)', text)
    return text

In [5]:
import os 

def clean_text_batch(batch):
    batch["article"] = [clean_text(text) for text in batch["article"]]
    batch["abstract"] = [clean_text(text) for text in batch["abstract"]]
    return batch

NUM_CPU = os.cpu_count()

cleaned_dataset = train_dataset.map(clean_text_batch, batched=True, num_proc=NUM_CPU)
cleaned_dataset.save_to_disk("train_dataset_cleaned")

Map (num_proc=4):   0%|          | 0/10151 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10151 [00:00<?, ? examples/s]

In [6]:
text = train_dataset['article'][0][:500]

print("Text before cleaning: \n")
print(text)
print("\n#####################\n")
print("Text after cleaning: \n")
print(clean_text(text))

Text before cleaning: 

additive models @xcite provide an important family of models for semiparametric regression or classification . some reasons for the success of additive models are their increased flexibility when compared to linear or generalized linear models and their increased interpretability when compared to fully nonparametric models .
it is well - known that good estimators in additive models are in general less prone to the curse of high dimensionality than good estimators in fully nonparametric models .

#####################

Text after cleaning: 

Additive models provide an important family of models for semiparametric regression or classification. some reasons for the success of additive models are their increased flexibility when compared to linear or generalized linear models and their increased interpretability when compared to fully nonparametric models. it is well - known that good estimators in additive models are in general less prone to the curse of high dime

In [13]:
import torch
import numpy as np

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

from peft import (
    LoraConfig, 
    get_peft_model, 
    TaskType
)

import nltk
from datasets import load_dataset, load_from_disk, DatasetDict
from datetime import datetime

In [16]:
train_dataset = load_from_disk("train_dataset_cleaned")
train_data_txt, validation_data_txt = train_dataset.train_test_split(test_size=0.1).values()
train_data_txt, validation_data_txt

(Dataset({
     features: ['article', 'abstract'],
     num_rows: 9135
 }),
 Dataset({
     features: ['article', 'abstract'],
     num_rows: 1016
 }))

In [7]:
base_model_name = 'facebook/bart-large-cnn'

base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)

# tokenization
# encoder_max_length = 256
# decoder_max_length = 64

encoder_max_length = 1024
decoder_max_length = 256

tokenizer = AutoTokenizer.from_pretrained(base_model_name, max_length=encoder_max_length, truncation=True)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
lora_config = LoraConfig(
    r=16,  
    lora_alpha=32,  
    lora_dropout=0.1,  
    bias="none",  
    task_type=TaskType.SEQ_2_SEQ_LM 
)

base_model_peft = get_peft_model(base_model, lora_config)
base_model_peft.print_trainable_parameters() 

trainable params: 2,359,296 || all params: 408,649,728 || trainable%: 0.5773


In [11]:
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch["article"], batch["abstract"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch

train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
    num_proc=NUM_CPU,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
    num_proc=NUM_CPU,
)

Map (num_proc=4):   0%|          | 0/9135 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1016 [00:00<?, ? examples/s]

In [14]:
folder_name = "preprocessed_dataset"
# train_data.save_to_disk(f"{folder_name}/train")
# validation_data.save_to_disk(f"{folder_name}/val")

train_data = load_from_disk(f"{folder_name}/train")
validation_data = load_from_disk(f"{folder_name}/val")

train_data, validation_data

(Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 9135
 }),
 Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 1016
 }))

In [13]:
train_data['input_ids'][0][:10], train_data['attention_mask'][0][:10], train_data['labels'][0][:10]

([0, 250, 467, 9, 33310, 23140, 19, 10, 1537, 22039],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 4688, 12547, 9, 7268, 1848, 5536, 33310, 16, 10])

In [8]:
import evaluate

nltk.download("punkt", quiet=True)

metric = evaluate.load("rouge")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    result = {key: value * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [11]:
# Custom function to prevent errors in Hugging Face code

def _custom_pad_tensors_to_max_len(self, tensor, max_length):
        if self.processing_class is not None and hasattr(self.processing_class, "pad_token_id"):
            # If PAD token is not defined at least EOS token has to be defined
            pad_token_id = (
                self.processing_class.pad_token_id if self.processing_class.pad_token_id is not None else self.processing_class.eos_token_id
            )
        else:
            if self.model.config.pad_token_id is not None:
                pad_token_id = self.model.config.pad_token_id
            else:
                raise ValueError("Pad_token_id must be set in the configuration of the model, in order to pad tensors")

        padded_tensor = pad_token_id * torch.ones(
            (tensor.shape[0], max_length), dtype=tensor.dtype, device=tensor.device
        )
        padded_tensor[:, : tensor.shape[-1]] = tensor
        return padded_tensor

Seq2SeqTrainer._pad_tensors_to_max_len = _custom_pad_tensors_to_max_len

In [16]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./lora_model",
    num_train_epochs=6,  
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=50,
    save_total_limit=3,
    report_to=["tensorboard"]
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=base_model_peft)

trainer = Seq2SeqTrainer(
    model=base_model_peft,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data, 
    processing_class=tokenizer,  
    compute_metrics=compute_metrics,
)

In [17]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 4.447519302368164,
 'eval_rouge1': 30.547,
 'eval_rouge2': 8.903,
 'eval_rougeL': 18.0604,
 'eval_rougeLsum': 27.0716,
 'eval_gen_len': 80.6555,
 'eval_runtime': 1126.7567,
 'eval_samples_per_second': 0.902,
 'eval_steps_per_second': 0.057}

In [18]:
trainer.train()

Step,Training Loss
50,4.550400
100,4.469200
150,4.415200
200,4.349900
250,4.270800
300,4.249900
350,4.244500
400,4.220800
450,4.193700
500,4.163900


TrainOutput(global_step=3426, training_loss=4.1341824083389485, metrics={'train_runtime': 13478.032, 'train_samples_per_second': 4.067, 'train_steps_per_second': 0.254, 'total_flos': 1.1957350239830016e+17, 'train_loss': 4.1341824083389485, 'epoch': 6.0})

In [19]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 4.017017841339111,
 'eval_rouge1': 42.4221,
 'eval_rouge2': 15.4355,
 'eval_rougeL': 24.1791,
 'eval_rougeLsum': 37.8461,
 'eval_gen_len': 133.9715,
 'eval_runtime': 1426.7385,
 'eval_samples_per_second': 0.712,
 'eval_steps_per_second': 0.045,
 'epoch': 6.0}

In [9]:
model_path = '/kaggle/working/lora_model/checkpoint-3426'
tuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [10]:
tuned_model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): lora.Linear(
              (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=1024, out_features=16, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=16, out_features=1024, bias=False)
              )
              (lora_embeddi

In [17]:
article, abstract = validation_data_txt[0].values()

inputs = tokenizer(article, padding=True, truncation=True, return_tensors="pt", max_length=encoder_max_length).input_ids
labels = tokenizer(abstract, padding=True, truncation=True, return_tensors="pt", max_length=encoder_max_length).input_ids

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"

inputs = inputs.to(device)
labels = labels.to(device)

base_model = base_model.to(device)
tuned_model = tuned_model.to(device)

In [19]:
base_outputs = base_model.generate(inputs, max_new_tokens=decoder_max_length, do_sample=False)
base_summary = tokenizer.decode(base_outputs[0], skip_special_tokens=True)

In [20]:
tuned_outputs = tuned_model.generate(inputs, max_new_tokens=decoder_max_length, do_sample=False)
tuned_summary = tokenizer.decode(tuned_outputs[0], skip_special_tokens=True)

In [21]:
base_outputs = base_outputs.to("cpu")
tuned_outputs = tuned_outputs.to("cpu")
labels = labels.to("cpu")

In [22]:
compute_metrics((base_outputs, labels)), compute_metrics((tuned_outputs, labels))

({'rouge1': 37.1585,
  'rouge2': 2.2099,
  'rougeL': 18.5792,
  'rougeLsum': 31.694,
  'gen_len': 82.0},
 {'rouge1': 54.7619,
  'rouge2': 23.2,
  'rougeL': 32.5397,
  'rougeLsum': 48.4127,
  'gen_len': 182.0})

In [23]:
base_summary

'The path - integral formulation of quantum mechanics is the foundation of many numerical methods that allow one to study with great accuracy the rich physics of interacting quantum systems. A number of important physical problems particularly in the fields of strongly correlated fermions and cold atoms can be fruitfully modeled by lattice hamiltonians. We propose a new method that generalizes and improves the approach of ref. ref.'

In [24]:
tuned_summary

'We present a generalization of the path - integral monte carlo (pimc) algorithm for lattice hamiltonians, based on continuous - time random walks, that is unaffected by time - step errors. we also introduce a worm - algorithm based method to calculate pure expectation values of arbitrary off - diagonal observables, which are generally out of the scope of existing lattice ground - state methods. the resulting algorithm naturally applies to fermions, using the fixed - node approximation, but a technique to improve systematically upon this approximation is proposed. our methodology is demonstrated by a few case studies on the one - dimensional heisenberg model and the two - dimensional fermion hubbard model. we show that the algorithm can be used to calculate the ground state energies of the fermionic hubbard models with a significantly better accuracy than that achieved by the fixed node approximation.'

In [6]:
# Obsolete version

import concurrent.futures
from datasets import Dataset, DatasetDict
from transformers import BartTokenizer
from tqdm import tqdm
import torch

tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

def split_text_into_chunks(text, max_tokens=1024):
    inputs = tokenizer(text, return_tensors="pt", truncation=False, padding=False)
    input_ids = inputs["input_ids"][0]

    chunks = []
    for i in range(0, len(input_ids), max_tokens):
        chunk = input_ids[i : i + max_tokens]
        chunks.append(tokenizer.decode(chunk, skip_special_tokens=True))
    return chunks

def prepare_chunked_dataset(dataset, max_article_tokens=1024, max_summary_tokens=150):
    chunked_articles = []
    chunked_summaries = []
    chunked_ids = []

    with concurrent.futures.ProcessPoolExecutor() as executor:
        article_chunks = list(tqdm(
            executor.map(split_text_into_chunks, dataset["article"], [max_article_tokens] * len(dataset["article"])), 
            total=len(dataset["article"]), desc="Chunking Articles"
        ))

        summary_chunks = list(tqdm(
            executor.map(split_text_into_chunks, dataset["abstract"], [max_summary_tokens] * len(dataset["abstract"])), 
            total=len(dataset["abstract"]), desc="Chunking Summaries"
        ))

    for idx, (art_chunks, sum_chunks) in enumerate(zip(article_chunks, summary_chunks)):
        for art_chunk, sum_chunk in zip(art_chunks, sum_chunks):
            chunked_articles.append(art_chunk)
            chunked_summaries.append(sum_chunk)
            chunked_ids.append(idx)  

    return Dataset.from_dict({"article": chunked_articles, "abstract": chunked_summaries, "example_id": chunked_ids})

def tokenize_function(examples):
    inputs = tokenizer(examples['article'], padding="max_length", truncation=True, max_length=1024)
    targets = tokenizer(examples['abstract'], padding="max_length", truncation=True, max_length=150)
    return {
        "input_ids": torch.tensor(inputs["input_ids"], dtype=torch.long), 
        "attention_mask": torch.tensor(inputs["attention_mask"], dtype=torch.long), 
        "labels": torch.tensor(targets["input_ids"], dtype=torch.long),
        "example_id": torch.tensor(examples["example_id"], dtype=torch.long)
    }

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

In [86]:
# Obsolete version

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    fp16=True,
    logging_dir="./logs",
    remove_unused_columns=False,
    report_to=['tensorboard']
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=base_model_peft)

trainer = Seq2SeqTrainer(
    model=base_model_peft,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    processing_class=tokenizer,  
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1 Fmeasure,Rouge2 Fmeasure,Rougel Fmeasure
1,3.123500,3.117505,0.360189,0.103463,0.205191
2,3.093400,3.116834,0.358605,0.103081,0.206600
3,3.070600,3.111456,0.362723,0.105114,0.207312


TrainOutput(global_step=1500, training_loss=3.095814697265625, metrics={'train_runtime': 4927.6089, 'train_samples_per_second': 2.435, 'train_steps_per_second': 0.304, 'total_flos': 2.6179201400832e+16, 'train_loss': 3.095814697265625, 'epoch': 3.0})